In [4]:
!pip3.12 install vnstock3 cffi

Defaulting to user installation because normal site-packages is not writeable
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)


In [10]:
from vnstock3 import Vnstock
stock = Vnstock().stock(symbol='ACB')

2024-12-12 13:03:54 - vnstock3.common.data.data_explorer - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2024-12-12 13:03:54 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [12]:
stock.listing.all_symbols()

SSLError: HTTPSConnectionPool(host='ai.vietcap.com.vn', port=443): Max retries exceeded with url: /api/get_all_tickers (Caused by SSLError(CertificateError("hostname 'ai.vietcap.com.vn' doesn't match either of '*.secureswiftcontent.com', 'secureswiftcontent.com'")))